In [6]:
%pip install pandas
import pandas as pd

# Load the CSV content into a DataFrame
#data = pd.read_csv('file/bquxjob_157d5c5d_1941367a86e.csv')
data = pd.read_csv('../test/datos/datos.csv')

# Display the first few rows of the DataFrame to understand its structure
data.head()


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: c:\Users\user\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install --upgrade pip


,Fecha,Plataforma,inversion,evento_objetivo,Instalaciones,cvr
0,2025-02-25,Google Ads,1245.29,177,796,0.222362
1,2025-02-25,Meta,1476.11,50,252,0.198413
2,2025-02-25,TikTok Ads,623.56,36,192,0.187500
3,2025-02-26,Google Ads,1307.30,268,720,0.372222
4,2025-02-26,Meta,966.41,95,190,0.500000


In [7]:
# Convert 'Fecha' to datetime format
data['Fecha'] = pd.to_datetime(data['Fecha'])

# Find the most recent date in the dataset
most_recent_date = data['Fecha'].max()

# Define the two periods for comparison
period_current_end = most_recent_date
period_current_start = period_current_end - pd.Timedelta(days=6)
period_previous_end = period_current_start - pd.Timedelta(days=1)
period_previous_start = period_previous_end - pd.Timedelta(days=6)

# Filter data for the two periods
df_current = data[(data['Fecha'] >= period_current_start) & (data['Fecha'] <= period_current_end)]
df_previous = data[(data['Fecha'] >= period_previous_start) & (data['Fecha'] <= period_previous_end)]

# Check the filtered data
df_current.head(), df_previous.head()

(        Fecha  Plataforma  inversion  evento_objetivo  Instalaciones       cvr
 24 2025-03-05  Google Ads     503.40              198            607  0.326194
 25 2025-03-05        Meta     591.15              113            234  0.482906
 26 2025-03-05  TikTok Ads     463.64               67            189  0.354497
 27 2025-03-06  Google Ads     500.78              172            556  0.309353
 28 2025-03-06        Meta     567.28               65            163  0.398773,
        Fecha  Plataforma  inversion  evento_objetivo  Instalaciones       cvr
 3 2025-02-26  Google Ads    1307.30              268            720  0.372222
 4 2025-02-26        Meta     966.41               95            190  0.500000
 5 2025-02-26  TikTok Ads     390.59               50            183  0.273224
 6 2025-02-27  Google Ads    1367.98              269            748  0.359626
 7 2025-02-27        Meta    1009.99               58            105  0.552381)

In [8]:
# Convert 'inversion' column to numeric
data['inversion'] = pd.to_numeric(data['inversion'], errors='coerce')
df_current['inversion'] = pd.to_numeric(df_current['inversion'], errors='coerce')
df_previous['inversion'] = pd.to_numeric(df_previous['inversion'], errors='coerce')

# Function to calculate metrics for a given period and platform
def calculate_metrics(df, platform):
    df_platform = df[df['Plataforma'] == platform]
    total_inversion = df_platform['inversion'].sum()
    total_eventos = df_platform['evento_objetivo'].sum()
    costo_medio = total_inversion / total_eventos if total_eventos != 0 else 0
    return total_inversion, costo_medio, total_eventos

# Get unique platforms
platforms = data['Plataforma'].unique()

# Calculate metrics for each platform in both periods
results = {
}
for platform in platforms:
    inv_prev, cost_prev, event_prev = calculate_metrics(df_previous, platform)
    inv_curr, cost_curr, event_curr = calculate_metrics(df_current, platform)
    results[platform] = {
        'inv_prev': inv_prev,
        'cost_prev': cost_prev,
        'event_prev': event_prev,
        'inv_curr': inv_curr,
        'cost_curr': cost_curr,
        'event_curr': event_curr,
        'cost_change_pct': ((cost_curr - cost_prev) / cost_prev * 100) if cost_prev != 0 else 0,
        'inv_change_pct': ((inv_curr - inv_prev) / inv_prev * 100) if inv_prev != 0 else 0
    }

results

C:\Users\user\AppData\Local\Temp\ipykernel_8044\3167008449.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_current['inversion'] = pd.to_numeric(df_current['inversion'], errors='coerce')
C:\Users\user\AppData\Local\Temp\ipykernel_8044\3167008449.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_previous['inversion'] = pd.to_numeric(df_previous['inversion'], errors='coerce')


{'Google Ads': {'inv_prev': np.float64(8517.39),
  'cost_prev': np.float64(5.530772727272727),
  'event_prev': np.int64(1540),
  'inv_curr': np.float64(3550.1),
  'cost_curr': np.float64(1.1316863245138666),
  'event_curr': np.int64(3137),
  'cost_change_pct': np.float64(-79.53836868158727),
  'inv_change_pct': np.float64(-58.31939126892157)},
 'Meta': {'inv_prev': np.float64(6064.719999999999),
  'cost_prev': np.float64(12.376979591836733),
  'event_prev': np.int64(490),
  'inv_curr': np.float64(4555.84),
  'cost_curr': np.float64(7.163270440251573),
  'event_curr': np.int64(636),
  'cost_change_pct': np.float64(-42.124244553363205),
  'inv_change_pct': np.float64(-24.87963170599796)},
 'TikTok Ads': {'inv_prev': np.float64(2272.0499999999997),
  'cost_prev': np.float64(7.889062499999999),
  'event_prev': np.int64(288),
  'inv_curr': np.float64(3241.2599999999998),
  'cost_curr': np.float64(3.538493449781659),
  'event_curr': np.int64(916),
  'cost_change_pct': np.float64(-55.14684476

In [9]:
for plt in results:
    print(f"Platform: {plt}")
    for key, value in results[plt].items():
        print(f"{key}: {value}")
    print()

Platform: Google Ads
inv_prev: 8517.39
cost_prev: 5.530772727272727
event_prev: 1540
inv_curr: 3550.1
cost_curr: 1.1316863245138666
event_curr: 3137
cost_change_pct: -79.53836868158727
inv_change_pct: -58.31939126892157

Platform: Meta
inv_prev: 6064.719999999999
cost_prev: 12.376979591836733
event_prev: 490
inv_curr: 4555.84
cost_curr: 7.163270440251573
event_curr: 636
cost_change_pct: -42.124244553363205
inv_change_pct: -24.87963170599796

Platform: TikTok Ads
inv_prev: 2272.0499999999997
cost_prev: 7.889062499999999
event_prev: 288
inv_curr: 3241.2599999999998
cost_curr: 3.538493449781659
event_curr: 916
cost_change_pct: -55.14684476410652
inv_change_pct: 42.65795206971678

